<address style="font-size: 1.17em; text-align: left;"><span style="font-size: xx-small;"> Actualizada el 15-03-2021.<br /></span></address>
<h3 style="font-size: 1.17em; text-align: center;">ESCUELA TÉCNICA SUPERIOR DE INGENIERÍA INFORMÁTICA</h3>
<h3 style="font-size: 1.17em; text-align: center;">UNIVERSIDAD DE SEVILLA</h3>
<h4 style="font-size: 1em; text-align: center;"><strong>ÁLGEBRA LINEAL Y NUMÉRICA</strong></h4>
<p> </p>
<h2>Eliminación gaussiana, pivoteo parcial, descomposiciones LU y de Cholesky</h2>
<p>En esta sesión práctica abordaremos los siguientes contenidos:</p>
<ul>
<li>Un repaso al proceso de <a href="#eliminacion">eliminación gaussiana</a> de una matriz $A$ y su aplicación a la resolución de sistemas de ecuaciones lineales. Mostraremos ejemplos numéricos de como el procedimiento de eliminación estándar puede provocar grandes errores  al trabajar con una precisión numérica finita.</li>
<li>Mostraremos dos modificaciones del proceso de eliminación gaussiana: el método con <a href="#pivoteo">pivoteo parcial</a> y el método con <a href="#pivoteope">pivoteo parcial escalado</a>. Ilustraremos con ejemplos como se pueden mejorar los errores provocados por el método estándar.</li>
<li>Se definirá la <a href="#desclu">descomposición $LU$</a> de una matriz $A$, aportando algoritmos para obtener tales descomposiciones bajo las técnicas de pivoteo estudiadas anteriormente.</li>
<li>Analizaremos la <strong>aplicación </strong>de la descomposición $LU$ a la <a href="#sislin">resolución de sistemas lineales</a> $Ax=b$.</li>
<li>Estudiaremos un caso particular de descomposición $LU$ cuando la matriz $A$ es <strong>simétrica y definida positiva</strong>, conocido como <a href="#cholesky">descomposición de Cholesky</a>.</li>
</ul>
<p>La práctica finaliza con la realización por parte del alumnado de un <a href="#cuestionario">cuestionario</a> personalizado.</p>

<p><a name="eliminacion"></a></p>
<div style="color: #000000; background-image: initial; background-attachment: initial; background-origin: initial; background-clip: initial; background-color: #ffffff; background-position: initial initial; background-repeat: initial initial; margin: 8px;">
<h3>Resolución de sistemas mediante eliminación gaussiana</h3>
<p>En la práctica anterior, definimos la función <span style="color: #0000ff;">forma_escalonada( A)</span> que nos permite obtener tanto las matrices $F$ y $U$ como la secuencia de transformaciones elementales por filas que reproducen el proceso seguido para escalonar la matriz $A$, según la relación $FA=U$. Admite las siguientes opciones y parámetros:</p>
<p style="text-align: center;"><span style="color: #0000ff;">F,U=forma_escalonada(A, canonica?,transformaciones?)</span></p>
<p>donde</p>
<ul>
<li><span style="color: #0000ff;">canonica?</span> puede ser <span style="color: #0000ff;">True</span> para forma escalonada canónica y <span style="color: #0000ff;">False</span> para forma escalonada simple</li>
<li><span style="color: #0000ff;">transformaciones?</span> puede ser <span style="color: #0000ff;">True </span>para que se proporcione la lista de transformaciones elementales filas que se han realizado y <span style="color: #0000ff;">False </span>para que se omitan.</li>
</ul>
</div>

>Ejecutar la celda que sigue para poder utilizar esta función.

In [1]:
load('codigo_P2.sage')   ## IMPRESCINDIBLE EJECUTAR esta celda para acceder a las funciones definidas.

<h3><span style="color: #0000ff;">Ejemplo 1</span></h3>
<p>Recordemos su uso con un ejemplo de escalonamiento de una matriz de números racionales, de orden 3x4:</p>

In [2]:
A = matrix(QQ, [[0, 1, -2, 1], [4, 1, 0, 3], [2, 1, 1, -2]]); show(A)

[ 0  1 -2  1]
[ 4  1  0  3]
[ 2  1  1 -2]

In [3]:
F, U = forma_escalonada(A, True, True)

Matriz original:


[ 0  1 -2  1]
[ 4  1  0  3]
[ 2  1  1 -2]

Transformaciones elementales realizadas:


<script type="math/tex">F_{1,2}</script>

<script type="math/tex">F_{3,1}(-\frac{1}{2})</script>

...

<script type="math/tex">F_{1,3}(-\frac{1}{4})</script>

<script type="math/tex">F_{2,3}(1)</script>

<script type="math/tex">F_{3}(\frac{1}{2})</script>

Forma escalonada canónica:


[  1   0   0 3/2]
[  0   1   0  -3]
[  0   0   1  -2]

In [4]:
F*A == U

True

<h4>Resolución de sistemas lineales</h4>
<p>Podemos aplicar esta función para la resolución de un sistema de ecuaciones lineales $Ax=b$, mediante escalonamiento de la matriz ampliada $\ M=(A|b)$.  El proceso se completa con la resolución del sistema a partir de la nueva matriz escalonada, mediante la técnica de <strong>sustitución regresiva</strong>.</p>
<p>La función <span style="color: #0000ff;">sustitucion_regresiva()</span> que definimos a continuación, aplicada sobre la matriz ampliada escalonada de un sistema de ecuaciones, devuelve su solución.</p>

In [5]:
def sustitucion_regresiva(U):
    m = U.ncols()
    n = U.nrows()
    # Se separa la ultima columna: U = ( M | x )
    M = U.submatrix(ncols=m-1) # Las m-1 primeras columnas (se quita la ultima)
    x = U.column(-1)           # La ultima columna.
    for k in range(n - 1, -1, -1):
        x[k] = (x[k] - sum(M[k, j]*x[j] for j in range(k + 1, n)))/M[k, k]
    return x

<h3 style="font-size: 1.17em;"><span style="color: #0000ff;">Ejemplo 2</span></h3>
<p>Se considera el sistema de ecuaciones $Ax=b$, donde</p>
<p>$$ A=\left(\begin{array}{rrrr} 8 & -2 & 0 & 7 \\ -4 & 5 & 6 & -22 \\ 2 & -2 & 5 & 5 \\ 2 & -1 & 1 & 3\end{array}\right) \ \ \ \ {\rm y} \ \ \ \ b=\left(\begin{array}{r}13 \\ -15 \\ 10 \\ 5\end{array}\right)$$</p>
<p>Resolver el sistema  siguiendo los siguientes procedimientos:</p>
<p>a) utilizando la función de SAGE  <span style="color: #0000ff;">solve_right(),</span></p>
<p>b) mediante eliminación gaussiana sobre la matriz ampliada del sistema,</p>
<p>c) a partir de la forma escalonada canónica de la matriz ampliada.</p>

In [6]:
A = matrix(QQ, [[8, -2, 0, 7], [-4, 5, 6, -22], [2, -2, 5, 5], [2, -1, 1, 3]]); show(A)

[  8  -2   0   7]
[ -4   5   6 -22]
[  2  -2   5   5]
[  2  -1   1   3]

In [7]:
b = vector(QQ, [13, -15, 10, 5]); show(b)

(13, -15, 10, 5)

<p>a) utilizando la función de SAGE  <span style="color: #0000ff;">solve_right():</span></p>

In [8]:
A.solve_right(b)

(1, 1, 1, 1)

<p>b) mediante eliminación gaussiana sobre la matriz ampliada del sistema:</p>

In [9]:
M = block_matrix([[A, b.column()]])
show(M)

[  8  -2   0   7| 13]
[ -4   5   6 -22|-15]
[  2  -2   5   5| 10]
[  2  -1   1   3|  5]

In [10]:
F, U = forma_escalonada(M)
show(U)

[     8     -2      0      7|    13]
[     0      4      6  -37/2| -17/2]
[     0      0   29/4 -59/16| 57/16]
[     0      0      0  -5/29| -5/29]

In [11]:
sustitucion_regresiva(U)

(1, 1, 1, 1)

<p>c) a partir de la forma escalonada canónica de la matriz ampliada:</p>

In [12]:
F,U = forma_escalonada(M, True)
show(U)

[1 0 0 0|1]
[0 1 0 0|1]
[0 0 1 0|1]
[0 0 0 1|1]

<p>Siendo las variables $x,y,z,t$,  la matriz ampliada del sistema se interpreta como:</p>
<p>$$\left\lbrace \begin{array}{ccccc} x&&&&=1\\ &y&&&=1\\ &&z&&=1 \\ &&&t&=1 \end{array}\right.$$</p>
<p>La solución se lee directamente en este sistema.</p>
<h3><a name="pivoteo"></a>Técnicas de pivoteo</h3>
<p>La resolución de un sistema de ecuaciones lineales mediante el método de eliminación gaussiana no está exenta de problemas debido a la transmisión de los errores de redondeo durante todo el proceso de cálculo. Veamos un ejemplo ilustrativo, trabajando con números reales en punto flotante.</p>
<h3><strong><span style="color: #0000ff;">Ejemplo 3</span></strong></h3>
<p>Consideremos el sistema lineal $\ Ax=b\ $ para las entradas</p>
<p>$ A=\left(\begin{array}{rrrr} 0.003 & 9.00 \\ 4.00 & 0.100 \end{array}\right) \ \ \ \ {\rm y} \ \ \ \ b=\left(\begin{array}{r}9.03 \\ 40.1\end{array}\right)$</p>
<p>cuya solución exacta es $x=\left(\begin{array}{r}10 \\ 1\end{array}\right)$</p>
<p>Lo resolvemos mediante escalonamiento simple sobre la matriz ampliada del sistema:</p>
<p> </p>

In [13]:
M = matrix([[0.003, 9.00, 9.03], [4.00, 0.100, 40.1]]); show(M)

[0.00300000000000000    9.00000000000000    9.03000000000000]
[   4.00000000000000   0.100000000000000    40.1000000000000]

In [16]:
F, U = forma_escalonada(M, False, True)

Matriz original:


[0.00300000000000000    9.00000000000000    9.03000000000000]
[   4.00000000000000   0.100000000000000    40.1000000000000]

Transformaciones elementales realizadas:


<script type="math/tex">F_{2,1}(-1333.33333333333)</script>

Forma escalonada simple:


[0.00300000000000000    9.00000000000000    9.03000000000000]
[  0.000000000000000   -11999.9000000000   -11999.9000000000]

In [14]:
sustitucion_regresiva(U)

(1, 1, 1, 1)

<p>Obsérvese que la solución se ve afectada por un pequeño error $(0.0000000000004)$, insignificante en este caso. No obstante, tales errores pueden magnificarse a medida que disminuye la precisión con la que realicemos los cálculos.</p>
<p>Vamos a repetir el proceso pero obligando a SAGE a realizar los cálculos con números reales en <span style="color: #000000;"><strong>punto flotante y 8 bits de precisión para la mantisa</strong></span>, lo que provocará grandes errores de redondeo al almacenar los datos:</p>

In [15]:
M8 = matrix(RealField(8), [[0.003, 9.00, 9.03], [4.00, 0.100, 40.1]])  ## RealField(8)  especifica 8 bits en la mantisa
show(M8)

[0.0030    9.0    9.0]
[   4.0   0.10    40.]

<p>Mostramos los valores numéricos almacenados en memoria, ligeramente diferentes de los mostrados en pantalla:</p>

In [16]:
n(M8)

[0.00300598144531250    9.00000000000000    9.00000000000000]
[   4.00000000000000   0.100097656250000    40.0000000000000]

In [17]:
F,U = forma_escalonada(M8, False, True)

Matriz original:


[0.0030    9.0    9.0]
[   4.0   0.10    40.]

Transformaciones elementales realizadas:


<script type="math/tex">F_{2,1}(-1300.)</script>

Forma escalonada simple:


[ 0.0030     9.0     9.0]
[   0.00 -12000. -12000.]

In [18]:
sustitucion_regresiva(U)

(21., 1.0)

<p>Obsérvese que ahora la solución obtenida $x=(21,1.0)$ dista mucho de la solución exacta $x=(10,1)$.</p>
<p>El problema viene motivado, fundamentalmente, porque la transformación elemental $F_{2,1}(-1300.)$ ha amplificado los errores de redondeo iniciales al multiplicar la primera fila por 1300.</p>
<p>Es preciso por tanto introducir alguna técnica que controle el tamaño de los multiplicadores que se emplean en las transformaciones elementales durante el proceso de escalonamiento. Vamos a estudiar, a continuación, variantes de eliminación gaussiana con estrategias de  elección de pivotes.</p>
<h4><strong>Eliminación gaussiana con pivoteo parcial</strong></h4>
<p>En el proceso de escalonamiento, en la etapa k-ésima, hasta ahora hemos tomado como pivote el primer elemento no nulo de la columna correspondiente, por debajo de la diagonal. Una variante consiste en buscar el elemento de esa parte de la columna que es mayor en valor absoluto a todos los demás e intercambiar las filas de forma que el elemento localizado pase a ser el pivote. De esta forma, al dividir por el nuevo pivote conseguimos que el multiplicador sea menor o igual que 1. Esta técnica se denomina eliminación gaussiana con <strong>pivoteo parcial</strong>.</p>
<p>Implentamos esta nueva estrategía. El método de Gauss  con pivoteo parcial, será realizado con</p>
<p style="text-align: center;"><span style="color: #0000ff;">forma_escalonada(M, algoritmo='pivoteo parcial')</span></p>

In [19]:
def busca_pivote_con_pivoteo_parcial(A, k, s):
    vmax = 0    
    (m, n) = A.dimensions()
    r = m
    while (s < n) and (vmax == 0):
        # buscamos el coef max (en valor absoluto) en la col s, a partir de la fila k y por debajo
        (vmax, r) = max( (abs(A[p, s]), p)  for p in range(k, m) )  
        if vmax == 0: s = s + 1 # sin pivote en esta columna, pasamos a la siguiente.
    return (r, s)    
    
busca_pivote['pivoteo parcial'] = busca_pivote_con_pivoteo_parcial

<h3><strong><span style="color: #0000ff;">Ejemplo 4</span></strong></h3>
<p>Volvemos sobre el  ejemplo anterior, para precisión de 8 bits en punto flotante pero con técnica de pivoteo parcial.</p>

In [20]:
M8 = matrix(RealField(8), [[0.003, 9.00, 9.03], [4.00, 0.100, 40.1]])
F, U = forma_escalonada(M8, False, True, algoritmo = 'pivoteo parcial')

Matriz original:


[0.0030    9.0    9.0]
[   4.0   0.10    40.]

Transformaciones elementales realizadas:


<script type="math/tex">F_{1,2}</script>

<script type="math/tex">F_{2,1}(-0.00075)</script>

Forma escalonada simple:


[ 4.0 0.10  40.]
[0.00  9.0  9.0]

In [21]:
sustitucion_regresiva(U)

(10., 1.0)

<p>¡Hemos obtenido la solución exacta!</p>
<p> No obstante, en la práctica pueden surgir nuevos contratiempos que no resuelve esta técnica de pivoteo parcial. El más característico es el <strong>problema del escalado </strong>de las filas de la matriz:</p>
<p>Supongamos que sustituimos una ecuación del sistema por un múltiplo grande de ella misma. Los elementos de la fila correspondiente se convierten, "artificialmente", en candidatos para pivotes en el método de pivoteo parcial ya que han crecido de tamaño, lo que puede generar nuevos problemas. Lo ilustraremos con el ejemplo anterior al multiplicar la primera fila por $10^4$ y repetir la técnica de pivoteo parcial.</p>
<h3><span style="color: #0000ff;">Ejemplo 5</span></h3>

In [22]:
M8 = matrix(RealField(8), [[0.003, 9.00, 9.03], [4.00, 0.100, 40.1]])
M8.rescale_row(0, 10^4)   # modificamos la primera fila; recuerda que Sage enumera comenzando por 0
show(M8)

[   30. 90000. 90000.]
[   4.0   0.10    40.]

In [23]:
F,U = forma_escalonada(M8, False, True, algoritmo = 'pivoteo parcial')

Matriz original:


[   30. 90000. 90000.]
[   4.0   0.10    40.]

Transformaciones elementales realizadas:


<script type="math/tex">F_{2,1}(-0.13)</script>

Forma escalonada simple:


[    30.  90000.  90000.]
[   0.00 -12000. -12000.]

In [24]:
sustitucion_regresiva(U)

(17., 1.0)

<p>Como se puede observar, hemos vuelto a obtener una solución totalmente errónea.</p>
<h4><a name="pivoteope"></a>Pivoteo parcial escalado</h4>
<p>Una nueva técnica de elección de pivote que intenta paliar este problema de escalado consiste en modificar el pivoteo parcial, tomando como nuevo pivote en la parte de columna correspondiente aquel que resulte mayor al dividir su valor absoluto por el mayor valor absoluto de los elementos de la fila que lo contiene. En lo sucesivo llamaremos a esta técnica como eliminación gaussiana con pivoteo parcial escalado.</p>
<p>Implementamos la nueva estrategía de busqueda del pivote y la utilizaremos con</p>
<p style="text-align: center;"><span style="color: #0000ff;">forma_escalonada(M, algoritmo='pivoteo parcial escalado')</span></p>

In [25]:
def busca_pivote_con_pivoteo_parcial_escalado(A, k, s):
    vmax = 0    
    (m, n) = A.dimensions()
    r = m
    while (s < n) and (vmax == 0):
        for p in range(k,m):
            rowmax = max( abs(A[p,j]) for j in range(s,n) )
            if rowmax != 0 and abs(A[p,s])/rowmax > vmax: 
                (vmax, r) = (abs(A[p,s]) / rowmax , p)
        if vmax == 0: s = s+1
    return (r, s)    
    
    
busca_pivote['pivoteo parcial escalado'] = busca_pivote_con_pivoteo_parcial_escalado

<h3><span style="color: #0000ff;">Ejemplo 6</span></h3>
<p>Volvemos sobre el <strong>ejemplo anterior, </strong>aplicando ahora la técnica de escalonamiento con pivoteo parcial escalado.</p>

In [26]:
F, U = forma_escalonada(M8, False, True, algoritmo = 'pivoteo parcial escalado')

Matriz original:


[   30. 90000. 90000.]
[   4.0   0.10    40.]

Transformaciones elementales realizadas:


<script type="math/tex">F_{1,2}</script>

<script type="math/tex">F_{2,1}(-7.5)</script>

Forma escalonada simple:


[   4.0   0.10    40.]
[  0.00 90000. 90000.]

In [27]:
sustitucion_regresiva(U)

(10., 1.0)

<p>Como puede observarse, una vez más, hemos paliado los efectos del problema de escalado, obteniendo la solución exacta.</p>

<h3><a name="desclu"></a><strong>Descomposición $LU$ de una matriz</strong></h3>
<p>El proceso de eliminación gaussiana de una matriz $A$, como hemos visto, nos lleva a una descomposición matricial del tipo $F\cdot A=U$, donde $U$ es una matriz triangular superior y $F$ es una matriz cuadrada no singular, cuya inversa llamaremos $L=F^{-1}$, resultando $A=L\cdot U$.</p>
<p>Por otra parte, el proceso de eliminación gaussiana sólo requiere a lo sumo dos tipos de transformaciones elementales: intercambios de filas, $F_{ij}\ $, y adición a una fila de un múltiplo de otra, $F_{i,j}(-m_{ij})$. Si podemos prescindir de intercambios de filas, resulta que $F$ y $L$ son <strong>matrices triangulares inferiores unitarias</strong> y, concretamente el elemento $l_{ij}$ de la parte inferior de la matriz $L$ viene dado por los multiplicadores utilizados en el escalonamiento, cambiados de signo; esto es, $l_{ij}=m_{ij}$.</p>
<p>En caso de que sea preciso o nos interese utilizar intercambio de filas, se puede obtener una descomposición del tipo $P\cdot A=L\cdot U$, donde ahora $P$ representa una matriz de permutación, cuyo efecto al multiplicar por la matriz $A$ consiste en reordenar las filas de la misma. Para obtener la matriz $P$ basta con guardar memoria de todos los intercambios de filas realizados a lo largo del proceso de eliminación y aplicar estos mismos intercambios a la matriz unidad.</p>
<p>Modificamos los algoritmos anteriores para que guarden memoria de las matrices $P$, $L$ y $U$. </p>
<p>A continuación se definen varias funciones que actúan sobre una matriz:</p>
<p><span style="color: #0000ff;">descompLU(A)</span> proporciona la descomposición LU de una matriz A en caso de que exista, informando en caso contrario. Nos devuelve la secuencia de las dos matrices de la descomposición:  L, U.</p>
<p><span style="color: #0000ff;">descompPLU(A)</span> proporciona la descomposición PLU de una matriz A resultante de la eliminación gaussiana con <span style="color: #0000ff;">pivoteo parcial</span>. Nos devuelve una secuencia de las tres matrices de la descomposición: P, L, U.</p>
<p><span style="color: #0000ff;">descompPLUppe(A)</span> proporciona la descomposición PLU de una matriz A resultante de la eliminación gaussiana con <span style="color: #0000ff;">pivoteo parcial escalado</span>. Nos devuelve una secuencia de las tres matrices de la descomposición: P, L, U.</p>

In [28]:
def tril(A, k = 0):
    L = copy(A)
    m = A.nrows()
    n = A.ncols()
    L = matrix(A.base_ring(),m, n, lambda i,j : 0 if j > i+k else A[i,j])
    return(L)

def triu(A, k = 0):
    return(tril(A.transpose(), k).transpose())

def descompLU(M):
    A = copy(M)
    m = A.nrows(); n = A.ncols()
    for k in range(m):
        if A[k, k] != 0:
            for t in range(k + 1, m):
                A[t, k] = A[t, k]/A[k, k]
                A[t, k+1:] = A[t, k+1:] - A[t, k]*A[k, k+1:]
        elif any( x!=0 for x in A[k+1:, k]):
            print('La matriz no admite descomposición LU sin intercambio de filas')
            return None, None
    L = identity_matrix(base_ring(A), m) + tril(A, -1)
    U = triu(A)
    return L, U

def descompPLU(M):
    'Descomposicion PLU de A. Se tiene PA=LU.'
    A = copy(M)
    m = A.nrows(); n = A.ncols()
    piv = list(range(n))
    for k in range(m):
        (vmax, q) = max(  (abs(A[s, k]), s) for s in range(k, m) )
        if q != k:
            piv[k], piv[q] = piv[q], piv[k]
            A.swap_rows(q, k)
        if A[k, k] != 0:
            for t in range(k + 1, m):
                A[t, k]    = A[t, k]/A[k, k]
                A[t, k+1:] = A[t, k+1:] - A[t, k]*A[k, k+1:]
    L = identity_matrix(base_ring(A), m) + tril(A, -1)
    U = triu(A)
    P = identity_matrix(base_ring(A), m).matrix_from_rows(piv)
    return P, L, U
        
        
def descompPLUppe(M):
    A = copy(M)
    m = A.nrows(); n = A.ncols()
    piv = list(range(n))
    for k in range(m):
        vmax = 0
        for s in range(k,m):
            mm = max( abs(A[s,j]) for j in range(k,n) )
            if mm!=0:
                v = abs(A[s, k])/mm
                if v > vmax: (vmax, q) = (v, s)
        if q != k:
            piv[k], piv[q] = piv[q], piv[k]
            A.swap_rows(q, k)
        if A[k, k] != 0:
            for t in range(k + 1, m):
                A[t, k] = A[t, k]/A[k, k]
                A[t, k+1:] = A[t, k+1:] - A[t, k]*A[k, k+1:]
    L = identity_matrix(base_ring(A), m) + tril(A, -1)
    U = triu(A)
    P = identity_matrix(base_ring(A), m).matrix_from_rows(piv)
    return P, L, U

<h3><span style="color: #0000ff;">Ejemplo 7</span></h3>
<p>Calculamos la descomposición $LU$ de una matriz $A$ de orden 3 y comparamos con el proceso de escalonamiento:</p>

In [29]:
A=matrix(QQ, [[1, 2, 3], [3, 24, 1], [-1, 5, 1]]); show(A)

[ 1  2  3]
[ 3 24  1]
[-1  5  1]

In [30]:
L, U = descompLU(A)
show(L)
show(U)

[   1    0    0]
[   3    1    0]
[  -1 7/18    1]

[   1    2    3]
[   0   18   -8]
[   0    0 64/9]

In [31]:
L*U == A

True

In [32]:
F, U1 = forma_escalonada(A, False, True)

Matriz original:


[ 1  2  3]
[ 3 24  1]
[-1  5  1]

Transformaciones elementales realizadas:


<script type="math/tex">F_{2,1}(-3)</script>

<script type="math/tex">F_{3,1}(1)</script>

<script type="math/tex">F_{3,2}(-\frac{7}{18})</script>

Forma escalonada simple:


[   1    2    3]
[   0   18   -8]
[   0    0 64/9]

<h3><span style="color: #0000ff;">Ejemplo 8</span></h3>
<p>Sobre el <strong>ejemplo anterior</strong>, calculamos las descomposiciones PLU con pivoteo parcial y parcial escalado:</p>

In [33]:
P, L, U = descompPLU(A)
show(P)
show(L)
show(U)

[0 1 0]
[0 0 1]
[1 0 0]

[    1     0     0]
[ -1/3     1     0]
[  1/3 -6/13     1]

[     3     24      1]
[     0     13    4/3]
[     0      0 128/39]

In [34]:
L*U == P*A

True

In [35]:
P, L, U = descompPLUppe(A)
show(P)
show(L)
show(U)

[1 0 0]
[0 1 0]
[0 0 1]

[   1    0    0]
[   3    1    0]
[  -1 7/18    1]

[   1    2    3]
[   0   18   -8]
[   0    0 64/9]

In [36]:
L*U == P*A

True

<h4><a name="sislin"></a>Aplicación de la descomposición $LU$ a la resolución de sistemas lineales</h4>
<p>Dado el sistema lineal $A\cdot x=b$ y la descomposición $L\cdot U=P\cdot A$, obtenemos las siguientes relaciones:</p>
<p>\begin{align*}A x = b &\Leftrightarrow& P A x = Pb \\&\Leftrightarrow& LU x = Pb\\&\Leftrightarrow& LU x = b_P\end{align*}</p>
<p> donde $b_P$ es una permutación del vector $b$. Podemos ahora separar el sistema lineal en dos subsistemas:</p>
<p>$$LU x = b_P \Leftrightarrow \left\lbrace \begin{array}{l} U x = g \\ L g = b_P\end{array}\right.$$</p>
<p>El primer sistema, $Ux=g$,  es triangular superior y se resuelve por sustitución regresiva de las variables $x_i$, aplicando para $i=n, i=n-1, \ldots, i=1$ (en este orden)</p>
<p>$$ x_i=\frac{g_i-\sum_{j=i+1}^{n}u_{ij}x_j}{u_{ii}}$$</p>
<p>El segundo sistema, triangular inferior, aunque también se podría resolver de forma similar, podemos interpretarlo y resolverlo a través de los multiplicadores que se han empleado durante el escalonamiento, ya que en realidad $g$ es el resultado de aplicar sobre $b_p$ las transformaciones del proceso de escalonamiento. Otra forma de verlo  es a partir de  la ecuación $\ \ L\cdot g=b_p$, de donde se obtiene que $\ \ g=L^{-1}\cdot b_p\ \ $ y $L^{-1}=F$ la matriz de las transformaciones elementales fila . Podemos obtener recursivamente $g$ de la siguiente forma:</p>
\begin{align*}
b_{p}^{(n)} & =g, \\
 & \text{y para $j$, $k$ con } 1 \le k < j \le n,\\
{ b_{p}}_{j}^{(k+1)} & ={ b_{p}}_{j}^{(k)}-m_{jk} { b_{p}}_{k}^{(k)} 
\end{align*}
<p>La función <span style="color: #0000ff;">modifica_lado_derecho($L, b_p$)</span> modifica el vector $b_p$ del lado derecho, obteniendo el vector $g$.</p>
<p>La función <span style="color: #0000ff;">sustitucion_regresiva2(U, g)</span> devuelve la solución $x$ del sistema. Esta función es similar a sustitucion_regresiva(U) con la única diferencia de que toma 2 parámetros por separado: la matriz U y el término de la derecha, el vector g, mientras que en la versión anterior se tomaba la matriz ampliada $(U|g)$.</p>

In [37]:
def modifica_lado_derecho(L, b):
    g = b.column()
    n = g.nrows()
    for k in range(n - 1):
        g[k+1:] = g[k+1:] - g[k,0]*L[k+1:, k]
    return g
    
        
def sustitucion_regresiva2(U, g):
    n = U.ncols()
    x = copy(g)
    for k in range(n - 1, -1, -1):
        x[k] = (g[k] - sum(U[k, j]*x[j] for j in range(k + 1, n)))/U[k, k]
    return x

<h4>Finalmente, la resolución de un sistema lineal $A\cdot x=b$ puede quedar reducida a tres fases:</h4>
<p>1) Descomposición $LU$ de la matriz $A$; si se hacen intercambio de filas (matriz $P$), entonces hay que actualizar el vector $b$ sustituyéndolo por el nuevo vector $b_p=P\cdot b$; en caso contrario, $b_p=b$.</p>
<p>2) Resolución del <span style="text-decoration: underline;"> sistema auxiliar</span>  $L\cdot g=b_p$, obteniendo el vector $g$. La resolución puede hacerse directamente como fase de modificación del lado derecho, de acuerdo a los multiplicadores empleados en las transformaciones elementales, con la función <span style="color: #0000ff;">modifica_lado_derecho</span>.</p>
<p>3) Sustitución regresiva en el sistema $U\cdot x=g$, con la función <span style="color: #0000ff;">sustitucion_regresiva2</span>.</p>
<h4>Interés de la descomposición $LU$ para la resolución de sistemas lineales:</h4>
<p>En ocasiones es preciso resolver sistemas lineales $Ax=b_i$ para una misma matriz $A$ y diferentes vectores $b_i$, con $i=1,2,...k$. No tiene sentido escalonar la matriz $A$ en cada caso sino que habrá que hacerlo una única vez y guardar el resultado para poder aplicarlo a cada vector $b_i$. Precisamente, la descomposición $LU$ o $PLU$, que solo depende de la matriz $A$, permite resolver los diferentes sistemas según el esquema de tres pasos descrito anteriormente. Esto es necesario sobre todo para valores $k$ muy grandes (muchos sistemas) o cuando los posibles vectores $b_i$ no se conocen a priori.</p>

<h3><span style="color: #0000ff;">Ejemplo 9</span></h3>
<p>Resolver, con ayuda de la descomposición LU, los sistemas de ecuaciones $Ax=b_i$ para los siguientes valores:</p>
<p>$ A=\left(\begin{array}{rrrr} 1 & 3 & -2 \\ 2 & -1 & 1\\ -1 & 2 & 0 \end{array}\right) \ \ \ \ {\rm y} \ \ \ \ b_1=\left(\begin{array}{r}2 \\ 2\\ 1\end{array}\right),\ \ b_2=\left(\begin{array}{r}4 \\ -1\\ 3\end{array}\right),\ \  b_3=\left(\begin{array}{r}0 \\ 1\\ 1\end{array}\right)$</p>
<p> </p>

In [38]:
A = matrix(QQ, [[1, 3, -2], [2, -1, 1], [-1, 2, 0]]); show(A)

[ 1  3 -2]
[ 2 -1  1]
[-1  2  0]

In [39]:
b1 = vector(QQ, [2, 2, 1])
b2 = vector(QQ, [4, -1, 3])
b3 = vector(QQ, [0, 1, 1])

In [40]:
L, U = descompLU(A)
show(L)
show(U)

[   1    0    0]
[   2    1    0]
[  -1 -5/7    1]

[   1    3   -2]
[   0   -7    5]
[   0    0 11/7]

In [45]:
for b in [b1, b2, b3]:
    show(b)
    g = modifica_lado_derecho(L, b)
    show(g)  # obtenemos la solución del sistema auxiliar
    z = sustitucion_regresiva2(U, g)    # obtenemos la solución final por sustitución regresiva.
    show(z)
    show(A*z)  # comprobamos la solución
    print('-----')

(2, 2, 1)

[   2]
[  -2]
[11/7]

[1]
[1]
[1]

[2]
[2]
[1]

-----


...

(0, 1, 1)

[   0]
[   1]
[12/7]

[ 3/11]
[ 7/11]
[12/11]

[0]
[1]
[1]

-----


<div style="color: #000000; background-image: initial; background-attachment: initial; background-origin: initial; background-clip: initial; background-color: #ffffff; background-position: initial initial; background-repeat: initial initial; margin: 8px;">
<h3 style="font-size: 1.17em;"><span style="color: #0000ff;">Ejemplo 10</span></h3>
<p>Resolver, con ayuda de la descomposición PLU, los sistemas de ecuaciones $Ax=b_i$ para los siguientes valores:</p>
<p>$ A=\left(\begin{array}{rrrr} 0 & 1 & -2 \\ 2 & -1 & 1\\ -1 & 2 & 0 \end{array}\right) \ \ \ \ {\rm y} \ \ \ \ b_1=\left(\begin{array}{r}2 \\ 1\\ 1\end{array}\right),\ \ b_2=\left(\begin{array}{r}4 \\ -1\\ 3\end{array}\right),\ \  b_3=\left(\begin{array}{r}-3 \\ 1\\ 1\end{array}\right)$</p>
</div>

In [41]:
A=matrix(QQ, [[0, 1, -2], [2, -1, 1], [-1, 2, 0]]); show(A)
b1 = vector(QQ,[2,1,1])
b2 = vector(QQ,[4,-1,3])
b3 = vector(QQ,[-3,1,1])

[ 0  1 -2]
[ 2 -1  1]
[-1  2  0]

In [42]:
L, U = descompLU(A)  # primero vamos a probar si existe descomposición LU

La matriz no admite descomposición LU sin intercambio de filas


In [43]:
P, L, U = descompPLU(A)
show(P)
show(L)
show(U)

[0 1 0]
[0 0 1]
[1 0 0]

[   1    0    0]
[-1/2    1    0]
[   0  2/3    1]

[   2   -1    1]
[   0  3/2  1/2]
[   0    0 -7/3]

In [44]:
P*A == L*U

True

In [50]:
for b in [b1, b2, b3]:
    show(b)
    bp = P*b
    show(bp)
    g = modifica_lado_derecho(L, bp)
    show(g)             # resolvemos el sistema auxiliar
    z = sustitucion_regresiva2(U, g)   #  obtenemos la solución final para el primer vector b propuesto
    show(z)
    show(A*z)           #  comprobamos que todo ha ido bien y la solución es correcta
    print('-----')

(2, 1, 1)

(1, 1, 2)

[  1]
[3/2]
[  1]

[ 9/7]
[ 8/7]
[-3/7]

[2]
[1]
[1]

...

(1, 1, -3)

[  1]
[3/2]
[ -4]

[-1/7]
[ 3/7]
[12/7]

[-3]
[ 1]
[ 1]

-----


<h3>Matrices regulares y diagonalmente dominantes</h3>
<p>Se dice que una matriz cuadrada $A$  es no singular, invertible o <strong>regular</strong> si tiene matriz inversa, esto es, existe otra matriz $B$ tal que $AB=BA=I$ (entonces $B$ es la matriz inversa de $A$ y se nota $A^{-1}$).</p>
<p>Desde el punto de vista computacional, el método de Gauss-Jordan es el más eficiente para hallar la inversa de una matriz dada. Así, una matriz es regular si y solo si su forma escalonada canónica es la matriz unidad.</p>
<p>Hay ocasiones donde se puede garantizar que una matriz es regular con un coste computacional menor. Tal es el caso de las matrices diagonalmente dominantes. Se dice que una matriz es <strong>diagonalmente dominante</strong> por filas  si el valor absoluto de cada elemento de la diagonal es mayor que la suma de los valores absolutos del resto de elementos de la misma fila. De manera similar, se define una matriz diagonalmente dominante por columnas.</p>
<p><strong>Teorema</strong></p>
<p>Toda matriz diagonalmente dominante por filas o por columnas es regular.</p>

<h3 style="font-size: 1.17em;"><span style="color: #0000ff;">Ejemplo 11</span></h3>
<p><span style="color: #0000ff;"><span style="color: #000000;">Comprobar que la siguiente matriz es diagonalmente dominante p<span style="color: #000000;">o</span></span><span style="color: #000000;">r</span><span style="color: #000000;"><span style="color: #000000;"> fila</span>s pero no lo es por columnas. Comprobar que es regular.</span><br /></span></p>

In [45]:
A = matrix(QQ, [[4, 1, -2], [2, -5, 1], [-3, 2, 6]]); show(A)

[ 4  1 -2]
[ 2 -5  1]
[-3  2  6]

In [46]:
F, U = forma_escalonada(A, True) # la matriz A tendrá inversa si su forma escalonada canónica es I

<p>En tal caso, la inversa de $A$ es $A^{-1}=F$.</p>

In [47]:
show(U)

[1 0 0]
[0 1 0]
[0 0 1]

In [48]:
show(F)

[ 32/121  10/121   9/121]
[ 15/121 -18/121   8/121]
[   1/11    1/11    2/11]

In [49]:
F*A

[1 0 0]
[0 1 0]
[0 0 1]

In [50]:
A*F

[1 0 0]
[0 1 0]
[0 0 1]

<h3><a name="cholesky"></a>Matrices simétricas y descomposición de Cholesky</h3>
<h4>Definiciones</h4>
<p>Una matriz $A$ se dice  <span style="color: #0000ff;">simétrica <span style="color: #000000;">si verifica que $\ A=A^t$</span></span> , es decir, coincide con su matriz traspuesta.</p>
<p>Una matriz simétrica se dice que es <span style="color: #0000ff;">definida positiva <span style="color: #000000;">si verifica que</span></span> para cualquier $x \in R^n$, distinto del vector nulo, $\ x^{t}Ax > 0$.</p>
<p> </p>
<h4><span style="background-color: #ffffff;">Teorema</span></h4>
<p><span style="background-color: #ffffff;">Toda matriz simétrica y definida positiva admite descomposición $LU$.</span></p>
<p> </p>
<p>En SAGE podemos comprobar si una matriz $A$ es simétrica mediante el atributo <span style="color: #0000ff;">is_symmetric()</span></p>

In [51]:
A = matrix(RDF, [[4, 2, 1], [2, 3, 0], [1, 0, 4]])
show(A)

[4.0 2.0 1.0]
[2.0 3.0 0.0]
[1.0 0.0 4.0]

In [52]:
A.is_symmetric()

True

<h4>Definición</h4>
Se dice que una matriz $A$ simétrica admite una descomposición de Cholesky si existe otra matriz $T=(t_{ij})$ triangular superior con $\forall i, t_{ii} > 0$ y $A=T^{t} T$.

<h4>Teorema</h4>
Sea $A$ una matriz simétrica. Las condiciones siguientes son equivalentes:
<ul>
<li>$A$ es definida positiva.</li>
<li>Todos los menores principales $A_k$ de $A$ son positivos.</li>
<li>$A$ admite descomposición de Cholesky.</li>
</ul>

<h4>Relación con la descomposición LU</h4>
Una matriz definida positiva siempre admite factorización $LU$.
<p>Sea $A=LU$ con</p>
<p>$$L=\left(\begin{array}{cccc}1 & 0 & \cdots & 0\\ l_{21} & 1 & \cdots & 0\\ \vdots & \vdots & \ddots & \vdots\\ l_{n1} & l_{n2} & \cdots & 1\end{array}\right), \qquad U=\left(\begin{array}{cccc}u_{11} & u_{12} & \cdots & u_{1n}\\ 0 & u_{22} & \cdots & u_{2n}\\ \vdots & \vdots & \ddots & \vdots\\ 0 & 0 & \cdots & u_{nn}\end{array}\right).$$</p>
<p>En $U$ extraemos de cada fila la raíz cuadrada del coeficiente diagonal de la fila. Da</p>
$$U=\left(\begin{array}{cccc}\sqrt{u_{11}} & 0 & \cdots & 0\\ 0 & \sqrt{u_{22}} & \cdots & 0\\ \vdots & \vdots & \ddots & \vdots\\ 0 & 0 & \cdots & \sqrt{u_{nn}} \end{array}\right)\left(\begin{array}{cccc}1 & \frac{u_{12}}{\sqrt{u_{11}}}& \cdots & \frac{u_{1n}}{\sqrt{u_{11}}} \\ 0 & 1 & \cdots & \frac{u_{2n}}{\sqrt{u_{22}}}\\ \vdots & \vdots & \ddots & \vdots\\ 0 & 0 & \cdots & 1 \end{array}\right).$$
Escribimos esto como
$$U=DV$$ 
con
$$D=\left(\begin{array}{cccc}\sqrt{u_{11}} & 0 & \cdots & 0\\ 0 & \sqrt{u_{22}} & \cdots & 0\\ \vdots & \vdots & \ddots & \vdots\\ 0 & 0 & \cdots & \sqrt{u_{nn}} \end{array}\right), \qquad 
V=\left(\begin{array}{cccc}1 & \frac{u_{12}}{\sqrt{u_{11}}}& \cdots & \frac{u_{1n}}{\sqrt{u_{11}}} \\ 0 & 1 & \cdots & \frac{u_{2n}}{\sqrt{u_{22}}}\\ \vdots & \vdots & \ddots & \vdots\\ 0 & 0 & \cdots & 1 \end{array}\right).$$

Tenemos, por tanto, $A=LDV=(LD)V=BV$ con $B=LD$ (triangular inferior), $V$ triangular superior. De la simetría de $A$ se deduce que $B=T^t$. 
<p> </p>
<h4 style="font-size: 1em;">Cálculo directo de la descomposición de Cholesky</h4>
<p> </p>
<div style="color: #000000; background-image: initial; background-attachment: initial; background-origin: initial; background-clip: initial; background-color: #ffffff; background-position: initial initial; background-repeat: initial initial; margin: 8px;">
<p>Partiendo de la descomposición $A=T^tT$, podemos plantear un sistema de ecuaciones con los elementos $t_{ij}$ como incógnitas:</p>
<p>$A=\left(\begin{array}{cccc}t_{11} & 0 & \cdots & 0\\ t_{21} & t_{22} & \cdots & 0\\ \vdots & \vdots & \ddots & \vdots\\ t_{n1} & t_{n2} & \cdots & t_{nn}\end{array}\right)\left(\begin{array}{cccc}t_{11} & t_{21} & \cdots & t_{n1}\\ 0 & t_{22} & \cdots & t_{n2}\\ \vdots & \vdots & \ddots & \vdots\\ 0 & 0 & \cdots & t_{nn}\end{array}\right)$</p>
<p>resultando las siguientes ecuaciones:</p>
<p>$$\forall i,\quad  t_{ii}=\sqrt{a_{ii}-\sum_{k=1}^{i-1}t_{ik}^2}$$</p>
<p>$$\forall i, j \text{ con $j < i$,} \quad t_{ij}=\frac1{t_{jj}}\left(a_{ij}-\sum_{k=1}^{j-1}t_{ik}t_{jk}\right)$$</p>
<p>La siguiente función <span style="color: #0000ff;">Cholesky(A)</span> proporciona la matriz $T$ de la descomposición de Cholesky $A=T^tT$, para una matriz $A$ simétrica y definida positiva.</p>
</div>
<p> </p>
</div>

In [53]:
def Cholesky(M):
    A = copy(M)
    R = A.base_ring()
    n = A.nrows()
    L = matrix(R, n, n, 0)
    if not A.is_symmetric():
        print('La matriz no es simétrica.')
        return None
    else:
        for i in range(n):
            for j in range(i):
                L[i,j]=1/L[j,j] * (A[i,j] - add(L[i,k]*L[j,k] for k in range(j)))
            aux = A[i,i] - sum(L[i,k]**2 for k in range(i))
            if aux <= 0:
                print('La matriz no es definida positiva. No admite descomposición de Cholesky.')
                return None
            L[i, i] = sqrt(aux)
        return L.transpose()

<h3><span style="color: #0000ff;">Ejemplo 12</span></h3>
<p>Dada la siguiente matriz</p>
<p>$$A=\left(\begin{array}{ccc}4 & 2 & 1\\2 &3 &0\\\ 1 & 0 & 4\end{array}\right)$$</p>
<p>se pide:</p>
<p>a) comprobar que es simétrica</p>
<p>b) demostrar que es definida positiva estudiando el signo de sus menores principales</p>
<p>c) demostrar que es definida positiva hallando su descomposición de Cholesky.</p>

In [54]:
### La matriz se elige de tipo SR (Symbolic Ring) para poder expresar 
### las raices cuadradas en el cálculo de la descomposición de Cholesky

A = matrix(SR, [[4, 2, 1], [2, 3, 0],[1, 0, 4]])  
show(A)

[4 2 1]
[2 3 0]
[1 0 4]

<p>a)</p>

In [55]:
A.is_symmetric()

True

<p>b)</p>

<p>El sguiente bucle va calculando los menores principales de la matriz $A$. Hay que elegir el orden adecuado de la matriz.</p>

In [56]:
orden = 3  ### orden de la matriz

for i in range(orden):
    menor_principal = A[:i + 1, :i + 1].det()
    if menor_principal >0   : signo = 'positivo'
    elif menor_principal < 0: signo = 'negativo'
    else                    : signo = 'nulo'
    print('orden: %i signo: %s.'%(i+1, signo))

orden: 1 signo: positivo.
orden: 2 signo: positivo.
orden: 3 signo: positivo.


<p>c)</p>

In [57]:
T = Cholesky(A)
show(T)

[             2              1            1/2]
[             0        sqrt(2)   -1/4*sqrt(2)]
[             0              0 1/2*sqrt(29/2)]

In [58]:
T.transpose()*T == A

True

<div style="color: #000000; background-image: initial; background-attachment: initial; background-origin: initial; background-clip: initial; background-color: #ffffff; background-position: initial initial; background-repeat: initial initial; margin: 8px;">
<h4>Coste computacional de la descomposición de Cholesky</h4>
<p>El coste computacional es aproximadamente del orden  $O(\frac 16 n^3)$ para la resolución del sistema anterior  en el caso de una matriz cuadrada de orden $n$, aunque se precisa calcular además $n$ raices cuadradas. Para $n$ grande, el coste es casi la mitad que el asociado a la descomposición $LU$.</p>
</div>

<div style="color: #000000; background-image: initial; background-attachment: initial; background-origin: initial; background-clip: initial; background-color: #ffffff; background-position: initial initial; background-repeat: initial initial; margin: 8px;">
<h4>Aplicación de la descomposición de Cholesky a la resolución de sistemas lineales</h4>
<p>Tal como sucediera con la descomposición $A=LU$ podemos aplicar la misma técnica para la descomposición $A=T^tT$ en caso de que $A$ sea simétrica y definida positiva, para resolver en dos etapas el sistema lineal $Ax=b$</p>
<p>Podemos ahora separar el sistema lineal en dos subsistemas:</p>
<p style="text-align: center;">$T\cdot x=g\ \ $  y  $\ \ T^t\cdot g=b$</p>
<p>El primer sistema es triangular superior y se resuelve por <strong>sustitución regresiva</strong> de las variables del vector $x$. El segundo sistema es triangular inferior y se resuelve, de manera similar, por <strong>sustitución progresiva</strong>. La fórmula que se aplica con $i=1$, $i=2 \ldots i=n$ (en este orden) es</p>
<p>$$ g_i=\frac{b_i-\sum_{j=1}^{i-1}t_{ij}g_j}{t_{ii}}$$</p>
<p>La siguiente función <span style="color: #0000ff;">sustitucion_progresiva2(L, b)</span> resuelve un sistema lineal con matriz L triangular inferior mediante sustitución progresiva</p>
</div>

In [59]:
def sustitucion_progresiva2(L, g):
    n = L.ncols()
    x = copy(g)
    for k in range(n):
        x[k] = (g[k] - sum(L[k, j]*x[j] for j in range(k)))/L[k, k]
    return x

<div style="color: #000000; background-image: initial; background-attachment: initial; background-origin: initial; background-clip: initial; background-color: #ffffff; background-position: initial initial; background-repeat: initial initial; margin: 8px;">
<h3 style="font-size: 1.17em;"><span style="color: #0000ff;">Ejemplo 13</span></h3>
<p>A partir de la matriz del <strong>ejemplo anterior</strong>, resolver el sistema lineal $Ax=b$ utilizando la descomposición de Cholesky, para </p>
<p>$$b=\left(\begin{array}{c}1 \\ 0 \\ 1 \end{array}\right)$$</p>
<p> </p>
</div>

In [60]:
A = matrix(SR, [[4, 2, 1], [2, 3, 0], [1, 0, 4]])
show(A)

[4 2 1]
[2 3 0]
[1 0 4]

In [61]:
b = vector(SR, [1, 0, 1]); b

(1, 0, 1)

In [62]:
T = Cholesky(A)
show(T)

[             2              1            1/2]
[             0        sqrt(2)   -1/4*sqrt(2)]
[             0              0 1/2*sqrt(29/2)]

In [63]:
g = sustitucion_progresiva2(T.transpose(), b)
show(g)         # resolvemos el primer sistema auxiliar por sustitución progresiva

(1/2, -1/4*sqrt(2), 5/58*sqrt(29/2))

In [64]:
z = sustitucion_regresiva2(T, g)  # resolvemos el segundo sistema por sustitución regresiva, obteniendo la solución final
z

(9/29, -6/29, 5/29)

In [65]:
show(A*z)
A*z == b #  comprobamos si las soluciones coinciden

(1, 0, 1)

True

<hr>

<h3><a name="cuestionario"></a></h3>
<h3>CUESTIONARIO PARA EL ALUMNADO</h3>

Introduzca en la variable NUMERO_CUESTIONARIO, que aparece más abajo, el número correspondiente a su cuestionario asignado y ejecuta la celda (*Shift* + *Enter* desde dentro de la celda). Accederá a un cuestionario con respuestas tipo test que se entregará cumplimentado antes de finalizar esta sesión práctica, siguiendo un modelo impreso que entregará el profesor y/o on-line en la plataforma de enseñanza virtual (seguir las instrucciones dadas en la clase de laboratorio por el docente).

**Ayuda para copiar los datos de los enunciados:**

* Se puede utilizar la función `matrix_from_copypaste` para copiar y pegar los coeficientes de las matrices. Selecciona con el ratón todos los datos del interior de la matriz y cópialos con Ctrl+C. Luego, pégalos (entre comillas) con Ctrl+V en el interior de la siguiente orden:
      A = matrix_from_copypaste(QQ, 3, 4, 'pegar_aqui_los_coeficientes' )   # matriz de orden 3x4 para datos en QQ.
* De forma análoga, se puede utilizar la función `vector_from_copypaste` para copiar y pegar los coeficientes de los vectores.
      b = vector_from_copypaste(QQ, 'pegar_aqui_los_coeficientes' )    # vector para datos en QQ.

In [66]:
NUMERO_CUESTIONARIO = 1   ### asignar un número natural entre 1 y 500

### No modificar el código de abajo

NOMBRE_FICHERO_EXAMEN = 'ALN_P2_t.htl'
load('show_html.sage')
load('codigo_examinar_html.sage')

if NUMERO_CUESTIONARIO>0:
    lector_examenes(NOMBRE_FICHERO_EXAMEN,NUMERO_CUESTIONARIO,False)

"1. (9.8, 0.98, -0.98)","2. (8.8, 3.0, -1.4)"
"3. (9.5, 0.90, -1.1)","4. (10., 1.0, -1.1)"
"1. (7.9, 6.0, -3.0)","2. (10., 0.55, -0.93)"
"3. (10., 0.91, -1.0)","4. (11., 1.2, 2.6)"
"1. (3, 2, -16, 60, 10)","2. (-10, 48, 306/7, 10, -9/7)"
"3. (34, -52/3, 43/7, -21/2, 5)","4. (-9, 1, 4, -8, -4)"
"1. (-2, 2, 1, -3, 2)","2. (3, -2, -1, -2, 1)"
3. El sistema no tiene solución.,"4. (2, -2, 3, -3, -3)"


In [75]:
A = matrix_from_copypaste(QQ, 3, 3, '−1.130,0.818,1.953,1.958,−0.180,1.050,2.361,−2.860,1.597' )  ## una matriz de orden 3x3 en QQ
show(A)

[ -113/100   979/500 2361/1000]
[  409/500     -9/50   -143/50]
[1953/1000     21/20 1597/1000]

In [83]:
B = vector_from_copypaste(QQ, '−12.000,11.000,19.000')  ## un vector en QQ
show(B)

(-12, 11, 19)

In [70]:
# Ejercicio 1

In [71]:
# Apartado 1
# Respuestas:
'''
1. (9.8, 0.98, -0.98)	2. (8.8, 3.0, -1.4)
3. (9.5, 0.90, -1.1)	4. (10., 1.0, -1.1)

−1.130,0.818,1.953,1.958,−0.180,1.050,2.361,−2.860,1.597,
'''

'\n1. (9.8, 0.98, -0.98)\t2. (8.8, 3.0, -1.4)\n3. (9.5, 0.90, -1.1)\t4. (10., 1.0, -1.1)\n'

In [85]:
A = matrix_from_copypaste(QQ, 3, 3, '−1.130,0.818,1.953,1.958,−0.180,1.050,2.361,−2.860,1.597')
show(A)

[ -113/100   979/500 2361/1000]
[  409/500     -9/50   -143/50]
[1953/1000     21/20 1597/1000]

In [86]:
B = vector_from_copypaste(QQ, '−12.000,11.000,19.000')
show(B)

(-12, 11, 19)

In [84]:
# Con precision 6 para la mantisa:
A = matrix(RealField(6), [[−1.130,0.818,1.953], [1.958,−0.180,1.050], [2.361,−2.860,1.597]])
show(A)
B = matrix(RealField(6), [[−12.000,11.000,19.000]])
show(B)

SyntaxError: invalid character in identifier (<ipython-input-84-aa8b51bd1914>, line 2)

In [79]:
# Escalonamiento simple sin pivoteo mas sustitución regresiva
M = block_matrix([[A,B.column()]])
show(M)

[ -113/100   979/500 2361/1000|      -12]
[  409/500     -9/50   -143/50|       11]
[1953/1000     21/20 1597/1000|       19]

In [80]:
F,U = forma_escalonada(M)
show(U)

[            -113/100              979/500            2361/1000|                 -12]
[                   0        349561/282500       -650251/565000|            1307/565]
[                   0                    0 3426275617/349561000|     -7011649/699122]

In [81]:
sustitucion_regresiva(U)

(34509054000/3426275617, 3144624750/3426275617, -3505824500/3426275617)

In [ ]:
# Apartado 2
# Respuestas:
'''
1. (7.9, 6.0, -3.0)	2. (10., 0.55, -0.93)
3. (10., 0.91, -1.0)	4. (11., 1.2, 2.6)
'''

In [87]:
# Escalonamiento simple con pivoteo parcial escalado más sustituión regresiva
F,U = forma_escalonada(M, False, True, algoritmo="pivoteo parcial escalado")
sustitucion_regresiva(U)

Matriz original:


[ -113/100   979/500 2361/1000|      -12]
[  409/500     -9/50   -143/50|       11]
[1953/1000     21/20 1597/1000|       19]

Transformaciones elementales realizadas:


<script type="math/tex">F_{1,3}</script>

<script type="math/tex">F_{2,1}(-\frac{818}{1953})</script>

...

<script type="math/tex">F_{2,3}</script>

<script type="math/tex">F_{3,2}(\frac{28820}{119297})</script>

Forma escalonada simple:


[             1953/1000                  21/20              1597/1000|                    19]
[                     0           119297/46500        6415643/1953000|            -1966/1953]
[                     0                      0 -3426275617/1252618500|       7011649/2505237]

(34509054000/3426275617, 3144624750/3426275617, -3505824500/3426275617)

In [ ]:
# Ejercicio 2

In [ ]:
# Apartado 1
# Respuestas:
'''
1. -1 2. 3 3. 9 4. 7
'''

In [88]:
A = matrix_from_copypaste(QQ, 3, 3, '81,−81,−54,−81,145,126,−54,126,198')
show(A)

[ 81 -81 -54]
[-81 145 126]
[-54 126 198]

In [89]:
B = vector_from_copypaste(QQ, '−324,388,288')
show(B)

(-324, 388, 288)

In [90]:
show(A.is_symmetric())

True

In [93]:
# Orden de la matriz A
orden = 3

for i in range(orden):
    menor_principal = A[:i + 1, :i + 1].det()
    if menor_principal >0   : signo = 'positivo'
    elif menor_principal < 0: signo = 'negativo'
    else                    : signo = 'nulo'
    print('orden: %i signo: %s.'%(i+1, signo))  

orden: 1 signo: positivo.
orden: 2 signo: positivo.
orden: 3 signo: positivo.


In [94]:
# Si todo se cumple: hallar la descomposicion y el elemento t2,3 de T
T = Cholesky(A)
show(T)

[ 9 -9 -6]
[ 0  8  9]
[ 0  0  9]

In [95]:
show(T.transpose()*T==A)

True

In [ ]:
# Si todo lo anterior es correcto: todos los ordenes positivos y la traspuesta es True entonces resolvemos:

In [ ]:
# Apartado 2
# Respuestas:
'''
1. [-36, 8, 0] 2. [-19, 1, -3] 3. [-18, -14, 0] 4. [-17, 22, 10]
'''

In [96]:
# Hallar la solucion del sistema auxiliar por sustitucion progresiva
G = sustitucion_progresiva2(T.transpose(),B)
show(G)

(-36, 8, 0)

In [97]:
# Apartado 3
# Respuestas:
'''
1. [-3, -3, -3] 2. [0, 0, -1] 3. [1, 0, 0] 4. [-3, 1, 0]
'''

In [98]:
# Hallar la solucion final del sistema Ax=b
Z = sustitucion_regresiva2(T,G)
show(Z)

(-3, 1, 0)

In [ ]:
# Ejercicio 3

In [ ]:
# Apartado 1
# Respuestas:
'''
1. La matriz es singular y no tiene ninguna propiedad más.
2. Es regular, simétrica y definida positiva, aunque no es diagonal dominante.
3. Tan solo es simétrica y singular.
4. Ninguna de las otras respuestas
'''

In [99]:
A = matrix_from_copypaste(QQ, 5, 5, '199,15,48,−47,−63,15,91,−84,−39,9,48,−84,170,55,−81,−47,−39,55,65,9,−63,9,−81,9,81')
show(A)

[199  15  48 -47 -63]
[ 15  91 -84 -39   9]
[ 48 -84 170  55 -81]
[-47 -39  55  65   9]
[-63   9 -81   9  81]

In [ ]:
# Se quiere estudiar si la matriz es regular, diagonal dominante, simétrica y def. positiva:

In [100]:
# Diagonal dominante:
F,U = forma_escalonada(A, True)

# A tiene inversa si su forma escalonada canonica es la identidad I
# Si F*A = A*F = I, diagonal dominante
# Una matriz es REGULAR si y solo si su forma escalonada canónica es la matriz unidad (esto es la U)
# REGULAR = NO SINGULAR
# SINGULAR = NO REGULAR
# Toda matriz diagonalmente dominante por filas o por columnas es regular pero no necesariamente al revés!!!!!!!
show(U)
show(F*A)
show(A*F)

[1 0 0 0 0]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 0 1 0]
[0 0 0 0 1]

[1 0 0 0 0]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 0 1 0]
[0 0 0 0 1]

[1 0 0 0 0]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 0 1 0]
[0 0 0 0 1]

In [ ]:
# Simetría:
show(A.is_symmetric())

In [101]:
# Orden de la matriz A
orden = 5

# Definida positiva: buscamos sus menores:
for i in range(orden):
    menor_principal = A[:i + 1, :i + 1].det()
    if menor_principal >0   : signo = 'positivo'
    elif menor_principal < 0: signo = 'negativo'
    else                    : signo = 'nulo'
    print('orden: %i signo: %s.'%(i+1, signo))

orden: 1 signo: positivo.
orden: 2 signo: positivo.
orden: 3 signo: positivo.
orden: 4 signo: positivo.
orden: 5 signo: positivo.


In [ ]:
# Ejercicio 4

In [ ]:
# Apartado 1
# Respuestas:
'''
1. La matriz  A  no admite ni descomposición LU ni descomposición PLU.
2. La matriz  A  admite descomposicion LU sin intercambio de filas.
3. La matriz  A  no admite descomposición LU porque es singular.
4. La matriz  A  no admite descomposicion LU sin intercambio de filas, pero sí admite descomposición PLU.
'''

In [111]:
A = matrix_from_copypaste(QQ, 5, 5, '3,−3,9,−6,21,3,−5,5,−2,9,−3,2,−10,11,−32,−1,−1,−8,1,−10,2,−4,1,−2,5')
show(A)

[  3   3  -3  -1   2]
[ -3  -5   2  -1  -4]
[  9   5 -10  -8   1]
[ -6  -2  11   1  -2]
[ 21   9 -32 -10   5]

In [112]:
B = vector_from_copypaste(QQ, '3,−1,−3,2,3')
show(B)

(3, -1, -3, 2, 3)

In [113]:
# ¿Existe descomposicion LU para A? 
L,U = descompLU(A)
show(L*U==A)

True

In [105]:
P,L,U = descompPLU(A)
show(L*U==P*A)

True

In [ ]:
# Apartado 2
# Respuestas:
'''
1. 1 2. 135/13 3. 39/7 4. -13/4
'''

In [106]:
# Hallar la matriz U de LU, si es posible, de lo contrario la matriz U de PLU con intercambio de filas. Indicar el valor U3,3
L,U = descompLU(A)
show(U)

[ 3  3 -3 -1  2]
[ 0 -2 -1 -2 -2]
[ 0  0  1 -1 -1]
[ 0  0  0 -2  1]
[ 0  0  0  0  0]

In [107]:
# Si no se puede LU, hacer PLU:
P,L,U = descompPLU(A)
show(U)

[    21      9    -32    -10      5]
[     0  -26/7  -18/7  -17/7  -23/7]
[     0      0  38/13 -58/13 -28/13]
[     0      0      0  -2/19   1/19]
[     0      0      0      0      0]

In [ ]:
# Apartado 3
# Respuestas:
'''
1. (3, 2, -16, 60, 10)	2. (-10, 48, 306/7, 10, -9/7)
3. (34, -52/3, 43/7, -21/2, 5)	4. (-9, 1, 4, -8, -4)
'''

In [114]:
# Determinar la solucion del sistema auxiliar con la descomposicion que se haya usado antes (LU o PLU):

# Comentar/descomentar o bien L,U o P,L,U en cada caso dependiendo de la que se uso en el apartado anterior!!!!!!!!
L,U = descompLU(A)
#P,L,U = descompPLU(A)

P*A == L*U
BP = P*B
G = modifica_lado_derecho(L,BP)
show(G)

[  3]
[  2]
[-16]
[ 61]
[ 11]

In [ ]:
# Apartado 4
# Respuestas:
'''
1. (-2, 2, 1, -3, 2)	2. (3, -2, -1, -2, 1)
3. El sistema no tiene solución.	4. (2, -2, 3, -3, -3)
'''

In [115]:
# Dar la solucion al sistema Ax=b ---- El sistema puede no tener solución, en cuyo caso el algoritmo falla, OJO!
Z = sustitucion_regresiva2(U,G)
show(Z)

ZeroDivisionError: rational division by zero